# Investigation of California Socioeconomic Relations Dataset

This contains the chapter on how we cleaned the dataset.

- [Requirements](#library-imports)
- [Cleaning the Data](#clean)

## Importing required libraries<a class="anchor" id="library-imports"></a>

In [1]:
# Standard python packages
import os
import sys

# Other package imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Cleaning the Data<a class="anchor" id="clean"></a>

Describe how we cleaned the data and removed unneccesary rows/columns etc.

In [10]:
## CODE GOES HERE